In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
noteevents = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/NOTEEVENTS.csv")
diagnoses = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/DIAGNOSES_ICD.csv")
diagnoses_dict = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/D_ICD_DIAGNOSES.csv")
admissions = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/ADMISSIONS.csv")
patients = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/PATIENTS.csv")


/usr/common/software/python/3.7-anaconda-2019.10/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
final_df = pd.read_pickle("pickles/final_df.pickle")

In [6]:
DOD = final_df.merge(admissions[['SUBJECT_ID',"HADM_ID","DEATHTIME"]],on = ['SUBJECT_ID',"HADM_ID"]).merge(diagnoses_dict[["ICD9_CODE","SHORT_TITLE"]],on="ICD9_CODE").merge(patients[["SUBJECT_ID", "DOD_HOSP", "DOD_SSN"]], on = "SUBJECT_ID")
DOD_inhos = DOD.dropna(subset=["DEATHTIME"]) #person not dead?
DOD_inhos = DOD_inhos[DOD_inhos["DISCHTIME"]==DOD_inhos["DEATHTIME"]] #sometimes typographical errors, so we have to check, but dischtime = deathtimme means inhos death
DOD_inhos=DOD_inhos.drop(["DOD_HOSP","DOD_SSN"], axis = 1)
DOD_inhos


,ROW_ID,SUBJECT_ID,HADM_ID,ICD9_CODE,DOB,ADMITTIME,DISCHTIME,DEATHTIME,SHORT_TITLE
9,1388,109,189332,33829,2117-08-07 00:00:00,2142-08-28 19:48:00,2142-08-30 15:20:00,2142-08-30 15:20:00,Chronic pain NEC
44,15700,1371,109490,32723,2125-08-05 00:00:00,2197-02-19 21:28:00,2197-03-15 20:57:00,2197-03-15 20:57:00,Obstructive sleep apnea
62,36446,3267,134370,32723,2138-03-06 00:00:00,2193-08-07 17:26:00,2193-08-28 12:04:00,2193-08-28 12:04:00,Obstructive sleep apnea
79,32545,2906,117044,32723,2097-04-01 00:00:00,2156-01-11 14:09:00,2156-01-30 11:50:00,2156-01-30 11:50:00,Obstructive sleep apnea
81,31800,2825,169889,32723,2116-03-29 00:00:00,2153-05-13 03:20:00,2153-05-14 01:50:00,2153-05-14 01:50:00,Obstructive sleep apnea
...,...,...,...,...,...,...,...,...,...
17304,476095,60987,116954,5713,2021-11-16 00:00:00,2103-08-13 18:03:00,2103-08-30 15:30:00,2103-08-30 15:30:00,Alcohol liver damage NOS
17345,398880,44521,118704,33822,2062-05-27 00:00:00,2136-09-24 18:14:00,2136-11-15 00:01:00,2136-11-15 00:01:00,Chron post-thoracot pain
17352,443994,54183,150129,30089,2108-06-04 00:00:00,2165-09-13 09:11:00,2165-09-27 21:10:00,2165-09-27 21:10:00,Somatoform disorders NEC
17358,638302,97128,110275,31230,2102-09-13 00:00:00,2185-04-04 15:13:00,2185-04-06 17:00:00,2185-04-06 17:00:00,Impulse control dis NOS


In [33]:
DOD_outhos_bool = DOD["DEATHTIME"].isnull() #check where deathtime not exist and dod_hosp exists = outhos death
DOD_outhos = DOD[DOD_outhos_bool]
DOD_outhos = DOD_outhos.dropna(subset = ["DOD_HOSP"]) #both deathtime and dod_hosp nan means not dead
DOD_outhos=DOD_outhos.drop("DEATHTIME",axis = 1)
# DOD_outhos = DOD_outhos.drop_duplicates(subset = ['SUBJECT_ID'],keep = "last")
DOD_outhos = DOD_outhos.sort_values(by = ['DISCHTIME']).drop_duplicates(subset = ['SUBJECT_ID'],keep= "last")
DOD_outhos

,ROW_ID,SUBJECT_ID,HADM_ID,ICD9_CODE,DOB,ADMITTIME,DISCHTIME,SHORT_TITLE,DOD_HOSP,DOD_SSN
16202,320104,28239,174102,30981,2036-05-31 00:00:00,2100-08-01 12:00:00,2100-08-06 16:00:00,Posttraumatic stress dis,2101-01-20 00:00:00,2101-01-20 00:00:00
10273,547172,76715,140337,311,2030-09-18 00:00:00,2100-09-01 18:20:00,2100-09-02 13:30:00,Depressive disorder NEC,2101-08-26 00:00:00,2101-08-26 00:00:00
8988,138011,12232,182524,311,1800-09-21 00:00:00,2100-09-21 19:00:00,2100-09-28 11:44:00,Depressive disorder NEC,2100-11-25 00:00:00,2100-11-25 00:00:00
7647,625678,94564,187397,5715,2043-01-01 00:00:00,2100-10-12 19:40:00,2100-10-19 18:45:00,Cirrhosis of liver NOS,2100-12-18 00:00:00,2100-12-18 00:00:00
15491,508403,68145,178575,3383,2035-02-21 00:00:00,2100-11-03 05:40:00,2100-11-16 15:58:00,Neoplasm related pain,2100-12-01 00:00:00,2100-12-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...
9220,256802,23039,110712,311,2143-08-20 00:00:00,2206-01-17 16:07:00,2206-01-20 17:40:00,Depressive disorder NEC,2206-06-13 00:00:00,NaN
535,251657,22588,188912,32723,2143-12-03 00:00:00,2206-02-11 18:09:00,2206-02-27 12:24:00,Obstructive sleep apnea,2206-03-03 00:00:00,2206-03-03 00:00:00
9202,230712,20656,120824,311,2131-12-18 00:00:00,2206-03-24 17:25:00,2206-03-28 13:19:00,Depressive disorder NEC,2206-09-01 00:00:00,NaN
142,41894,3792,176637,32723,2147-06-23 00:00:00,2206-10-05 04:24:00,2206-10-11 16:06:00,Obstructive sleep apnea,2206-10-20 00:00:00,NaN


In [34]:
DOD_outhos["SHORT_TITLE"].value_counts()

Depressive disorder NEC     303
Cirrhosis of liver NOS      210
Tobacco use disorder        196
Obstructive sleep apnea     182
Anxiety state NOS           128
                           ... 
Poisoning-methadone           1
Nonpsychot brain syn NOS      1
Undeterm pois-psychotrop      1
Poisoning-sympatholytics      1
Chronic postop pain NEC       1
Name: SHORT_TITLE, Length: 123, dtype: int64

In [11]:
DOD_outhos[DOD_outhos["SHORT_TITLE"]=="Depressive disorder NEC"]

,ROW_ID,SUBJECT_ID,HADM_ID,ICD9_CODE,DOB,ADMITTIME,DISCHTIME,SHORT_TITLE,DOD_HOSP,DOD_SSN
136,41012,3703,179102,311,2086-04-14 00:00:00,2173-08-12 05:49:00,2173-08-15 18:15:00,Depressive disorder NEC,2173-08-28 00:00:00,2173-08-28 00:00:00
137,41030,3703,185494,311,2086-04-14 00:00:00,2172-04-11 16:24:00,2172-04-20 16:13:00,Depressive disorder NEC,2173-08-28 00:00:00,2173-08-28 00:00:00
202,101341,9054,165488,311,2031-08-15 00:00:00,2103-09-17 16:25:00,2103-10-05 19:00:00,Depressive disorder NEC,2103-10-19 00:00:00,2103-10-19 00:00:00
358,193984,17399,107705,311,2034-09-14 00:00:00,2112-02-29 16:52:00,2112-03-04 15:25:00,Depressive disorder NEC,2112-07-31 00:00:00,2112-07-31 00:00:00
372,180179,16194,178564,311,2096-04-28 00:00:00,2166-12-11 23:21:00,2166-12-17 16:25:00,Depressive disorder NEC,2168-01-13 00:00:00,2168-01-13 00:00:00
...,...,...,...,...,...,...,...,...,...,...
10675,647734,99312,134467,311,2108-10-28 00:00:00,2157-04-24 06:49:00,2157-05-03 17:36:00,Depressive disorder NEC,2157-05-08 00:00:00,2157-05-08 00:00:00
10680,646514,99063,166460,311,2090-02-28 00:00:00,2169-10-11 16:29:00,2169-10-19 16:00:00,Depressive disorder NEC,2169-11-02 00:00:00,2169-11-02 00:00:00
10684,632999,96104,152774,311,2025-03-10 00:00:00,2111-08-24 16:19:00,2111-09-05 05:15:00,Depressive disorder NEC,2111-09-21 00:00:00,2111-09-21 00:00:00
10697,650828,99937,129380,311,2062-09-06 00:00:00,2128-05-11 15:16:00,2128-05-20 13:35:00,Depressive disorder NEC,2129-05-18 00:00:00,NaN
